In [1]:
import os
import time
import datetime as dt
from datetime import datetime
from multiprocessing import Pool
from recipe_scrapers import scrap_me

from time import sleep, time
from random import randint, choice
import requests
import urllib
from bs4 import BeautifulSoup
import csv
import re

In [2]:
# OS
NOW           = dt.datetime.now()
FILE_NAME     = 'recipe.csv'
DATASET_FOLDER = 'input/csv_files/'
IMGS_FOLDER  = 'input/images/search_thumbnails/'

# Chefkoch.de Seite
CHEFKOCH_URL  = 'http://www.chefkoch.de'
START_URL     = 'http://www.chefkoch.de/rs/s'
CATEGORY      = '/Rezepte.html'

In [3]:
desktop_agents = ['Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36',
                 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36',
                 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36',
                 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1) AppleWebKit/602.2.14 (KHTML, like Gecko) Version/10.0.1 Safari/602.2.14',
                 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36',
                 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.98 Safari/537.36',
                 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.98 Safari/537.36',
                 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36',
                 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36',
                 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0']

def random_headers():
    return {'User-Agent': choice(desktop_agents),'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'}

In [4]:
def _get_html(url):
    page = ''
    while page == '':
        try:
            page = requests.get(url, headers=random_headers())
        except:
            print('Connection refused')
            time.sleep(10)
            continue
    return page.text



In [16]:
from recipe_scrapers import scrap_me
import urllib.request
from random import randint 

num = 1

def _scrape_instructions(raw_link):
    link = _get_html(raw_link)
    scrap = scrap_me(raw_link)
    
    data = {'title' : scrap.title(),
    'preparation_time' : scrap.total_time(),
    'img_link' : _get_images(link),
    'ingredients' : scrap.ingredients()}
    
    if(data['ingredients'] == None or data['img_link']  == None):
        return
    
    img_path = os.path.join(os.getcwd(),'img')
    download_path = os.path.join(img_path,data['title'])
    if not os.path.exists(download_path):
        os.makedirs(download_path)
    try:
        urllib.request.urlretrieve(str(data['img_link']), download_path + '/'  + str(randint(1,10000)) + '.jpg' ) #download img to specific folder
    except:
        pass
    _write_to_recipes(data)

    
def _write_to_recipes(data):
    path = os.path.join(os.getcwd(),'food.csv')
    with open(path, 'a') as f:
        writer = csv.writer(f)
        try:
            writer.writerow((data['title'],
                            data['preparation_time'],
                            data['img_link'],
                            data['ingredients']))
        except:
            pass
    f.close()

        
def _get_images(html):
    soup = BeautifulSoup(html, 'lxml')
    try:
        img = soup.find('picture', class_='photo-wrap').find('source')['srcset']
        return(img)
    except:
        pass
    
    
def _get_ingredients(html):
    soup = BeautifulSoup(html, 'lxml')
    try:
        #all_ingredients = soup.find('blockquote').find('p').text.split('\n')
        return(all_ingredients)
    except:
        pass
    #print( tot)



def _get_links(html):
    soup = BeautifulSoup(html, 'lxml')
    links = soup.find_all('a', class_='show-quick-view')#get all links
    for individual in links:
        href = 'https://www.epicurious.com' + individual['href']
        _scrape_instructions(href)

for page in range(1,2357): #total page count in food republc
    link = 'https://www.epicurious.com/search/?page=' + str(page)
    _get_links(_get_html(link))
    
    
    

KeyboardInterrupt: 

In [44]:
start_time = time()


'''
all_links = _get_links(_get_html('https://www.weightwatchers.com/us/recipe'))
(_get_html(all_links[5]))

for link in range(3,len(all_links)):
    sub = _get_links(_get_html(all_links[link]))
    for individual in sub:
        _get_data(_get_html(individual))
    #print(len(_get_images(_get_html(link))))
'''

print("--- %s seconds ---" % (time() - start_time))

--- 9.012222290039062e-05 seconds ---
